In [1]:
# Imports
import requests  # send request
from bs4 import BeautifulSoup  # parse web pages
import pandas as pd  # csv
from time import sleep  # wait
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm  # 导入 tqdm
import json

In [11]:
headers = {
    'authority': 'arxiv.org',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'cache-control': 'no-cache',
    'sec-ch-ua': '"Not A(Brand";v="99", "Microsoft Edge";v="121", "Chromium";v="121"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0',
}

base_url = "https://arxiv.org/list/cs/pastweek?skip=0&show=25"
total_papers = 100  # 想要爬取的总论文数
papers_per_page = 25  # 每页显示的论文数
num_pages = int(total_papers/papers_per_page)
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')

In [12]:
title_list = []
author_list = []
subject_list = []
abstract_list = []

In [13]:
def extract(soup, headers):
    articles = soup.find_all('div', class_='meta')
    for article in articles:
        # 提取标题并去除 "Title:"
        title = article.find('div', class_='list-title mathjax').text.strip()
        title = title.replace('Title:', '').strip()
        title_list.append(title)
        
        # 提取作者
        author = article.find('div', class_='list-authors').text.strip()
        author = author.replace('Authors:', '').strip()
        author_list.append(author)
        
        # 提取主题
        subject = article.find('div', class_='list-subjects').text.strip()
        subject = subject.replace('Subjects:', '').strip()
        subject_list.append(subject)

    links = soup.find_all('a', title='Abstract')

    for link in tqdm(links, desc="提取进度"):  # 使用 tqdm 显示提取进度
        paper_url = "https://arxiv.org" + link['href']  # 构造完整的论文链接
        paper_page = requests.get(paper_url, headers=headers)
        paper_soup = BeautifulSoup(paper_page.content, 'html.parser')
        abstract = paper_soup.find('blockquote', class_='abstract').text.strip()  # 提取摘要内容
        abstract = abstract.replace('Abstract:', '').strip()
        abstract_list.append(abstract)
        
    return title_list, author_list, subject_list, abstract_list

In [14]:
for page_num in range(num_pages):
    url = f"{base_url}&skip={page_num * 25}"
    response = requests.get(url)
    if response.status_code == 200: 
        soup = BeautifulSoup(response.content, 'html.parser')
        extract(soup, headers)
    else:
        print(f"Failed to fetch page {page_num + 1}")

提取进度: 100%|██████████████████████████████████████████| 25/25 [00:29<00:00,  1.19s/it]


In [15]:
data = {'Title': title_list,'Subject': subject_list,'Author': author_list,'Abstract': abstract_list}
df = pd.DataFrame(data)
df.to_csv('output.csv', index=False)